# Paper: Tracking Error Rebalancing
http://www.iijournals.com/doi/abs/10.3905/jpm.2011.37.4.054?journalCode=jpm

<img src="files/tracking_error_opt.png">

In [1]:
import pandas as pd
import numpy as np
import collections

import math
import cvxpy

In [2]:
def corr2cov(corr, std):
    '''convert correlation matrix to covariance matrix given standard deviation

    Parameters
    ----------
    corr : array_like, 2d
        correlation matrix, see Notes
    std : array_like, 1d
        standard deviation

    Returns
    -------
    cov : ndarray (subclass)
        covariance matrix

    Notes
    -----
    This function does not convert subclasses of ndarrays. This requires
    that multiplication is defined elementwise. np.ma.array are allowed, but
    not matrices.

    '''
    corr = np.asanyarray(corr)
    std_ = np.asanyarray(std)
    cov = corr * np.outer(std_, std_)
    return cov

# Example data from the paper

In [3]:
d = collections.OrderedDict()

In [4]:
d = {'us_equity': {'weight': 0.39, 'policy': 0.45, 'expected_risk': 0.1975, 'costs': 0.003},
     'non_us_equity': {'weight': 0.158, 'policy': 0.2, 'expected_risk': 0.24, 'costs': 0.005},
     'fixed_income': {'weight': 0.293, 'policy': 0.2, 'expected_risk': 0.0525, 'costs': 0.025},
     'tips': {'weight': 0.068, 'policy': 0.05, 'expected_risk': 0.0975, 'costs': 0.003},
     'privaty_equity': {'weight': 0.046, 'policy': 0.05, 'expected_risk': 0.24, 'costs': 1.0},
     'real_state': {'weight': 0.043, 'policy': 0.05, 'expected_risk': 0.37, 'costs': 0.003}
    }

In [5]:
correlation = np.array([[1.0,0.95,0.15,0.45,0.95,0.8],
                       [0.95,1.0,0.3,0.55,0.85,0.7],
                       [0.15,0.3,1.0,0.8,0.15,0.2],
                       [0.45,0.55,0.8,1.0,0.45,0.5],
                       [0.95,0.85,0.15,0.45,1.0,0.9],
                       [0.8,0.7,0.2,0.5,0.9,1.0]])

In [6]:
correlation

array([[ 1.  ,  0.95,  0.15,  0.45,  0.95,  0.8 ],
       [ 0.95,  1.  ,  0.3 ,  0.55,  0.85,  0.7 ],
       [ 0.15,  0.3 ,  1.  ,  0.8 ,  0.15,  0.2 ],
       [ 0.45,  0.55,  0.8 ,  1.  ,  0.45,  0.5 ],
       [ 0.95,  0.85,  0.15,  0.45,  1.  ,  0.9 ],
       [ 0.8 ,  0.7 ,  0.2 ,  0.5 ,  0.9 ,  1.  ]])

In [7]:
df = pd.DataFrame.from_dict(d, orient='index')
df = df.reindex(index=['us_equity', 'non_us_equity', 'fixed_income', 'tips', 'privaty_equity', 'real_state'])

In [8]:
df

,expected_risk,costs,weight,policy
us_equity,0.1975,0.003,0.390,0.45
non_us_equity,0.2400,0.005,0.158,0.20
fixed_income,0.0525,0.025,0.293,0.20
tips,0.0975,0.003,0.068,0.05
privaty_equity,0.2400,1.000,0.046,0.05
real_state,0.3700,0.003,0.043,0.05


In [9]:
cov_m = corr2cov(correlation, df['expected_risk'])

In [10]:
cov_m

array([[ 0.03900625,  0.04503   ,  0.00155531,  0.00866531,  0.04503   ,
         0.05846   ],
       [ 0.04503   ,  0.0576    ,  0.00378   ,  0.01287   ,  0.04896   ,
         0.06216   ],
       [ 0.00155531,  0.00378   ,  0.00275625,  0.004095  ,  0.00189   ,
         0.003885  ],
       [ 0.00866531,  0.01287   ,  0.004095  ,  0.00950625,  0.01053   ,
         0.0180375 ],
       [ 0.04503   ,  0.04896   ,  0.00189   ,  0.01053   ,  0.0576    ,
         0.07992   ],
       [ 0.05846   ,  0.06216   ,  0.003885  ,  0.0180375 ,  0.07992   ,
         0.1369    ]])

In [11]:
h1_b = df['weight']-df['policy']
h1_b.values

array([-0.06 , -0.042,  0.093,  0.018, -0.004, -0.007])

Replicating tracking error of the example: 2.3%

In [12]:
print(math.sqrt(np.dot(np.dot(h1_b.values,cov_m),h1_b.values)))

0.02340611648159515


# Extracting variables and parameters for the optimization

In [13]:
n = len(df['weight'].values)

In [14]:
h1 = cvxpy.Variable(n)

In [15]:
h0 = df['weight'].values

In [16]:
b = df['policy'].values

In [17]:
V = cov_m

In [18]:
TC =  df['costs'].values

In [19]:
obj_func = cvxpy.Minimize(cvxpy.abs(h1-h0).T*TC)

In [20]:
c2 = cvxpy.sum_entries(h1) == 1.0

In [21]:
c3 = (h1 >= 0)

In [22]:
# Problem with the next constraint. It looks like its not convex.

In [23]:
#c1 = (cvxpy.sqrt( ((h1-b).T*V)*h1-b ) <= 0.01)

In [24]:
# another way of modeling c1: x.TPx (quad_form)
# http://www.cvxpy.org/en/latest/tutorial/functions/index.html#scalar-functions

In [45]:
c1 = (cvxpy.quad_form(h1-b, V)  <= 0.01**2)

In [46]:
prob = cvxpy.Problem(obj_func, [c1, c2, c3])

In [47]:
prob.solve()

0.00046098404150142746

In [48]:
print(h1.value)

[[  4.10738034e-01]
 [  1.81491983e-01]
 [  2.93000000e-01]
 [  2.29565026e-08]
 [  4.60000000e-02]
 [  6.87699610e-02]]


In [44]:
print(np.dot(abs(h1.value-h0).T,TC))

[[ 0.35174259  0.11499825  0.25154159  0.02742825  0.00605838  0.00915738]]


In [34]:
TC

array([ 0.003,  0.005,  0.025,  0.003,  1.   ,  0.003])

In [49]:
x = h1.value

In [50]:
x

matrix([[  4.10738034e-01],
        [  1.81491983e-01],
        [  2.93000000e-01],
        [  2.29565026e-08],
        [  4.60000000e-02],
        [  6.87699610e-02]])

In [51]:
TC

array([ 0.003,  0.005,  0.025,  0.003,  1.   ,  0.003])

In [53]:
tc2 = np.copy(TC)

In [54]:
tc2[4] = 0

In [55]:
tc2

array([ 0.003,  0.005,  0.025,  0.003,  0.   ,  0.003])

In [60]:
diff = abs(x-h0)

In [62]:
h0

array([ 0.39 ,  0.158,  0.293,  0.068,  0.046,  0.043])

In [65]:
diff = abs(x - df['weight'].values)

In [68]:
x.transpose()

matrix([[  4.10738034e-01,   1.81491983e-01,   2.93000000e-01,
           2.29565026e-08,   4.60000000e-02,   6.87699610e-02]])

In [69]:
x.transpose()-h0

matrix([[  2.07380339e-02,   2.34919826e-02,  -4.02100742e-10,
          -6.79999770e-02,  -8.00054467e-14,   2.57699610e-02]])

In [73]:
np.dot(abs((x.transpose()-h0)),tc2)

matrix([[ 0.00046098]])

In [ ]:
0.00046098